<small><small><i>
All the IPython Notebooks in this **Python Decision Tree and Random Forest** series by Dr. Milaan Parmar are available @ **[GitHub](https://github.com/milaan9/Python_Decision_Tree_and_Random_Forest)**
</i></small></small>

In [46]:
import sys
sys.path.append('..\\models')

import pandas as pd
from sklearn.model_selection import train_test_split # Import train_test_split function
from matplotlib import pyplot as plt
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation
import numpy as np

from ID3 import ID3
from NBC_Categorical import NBC_Categorical
from datasets import get_airline_dataset, get_exams_dataset, get_ecommerce_dataset

## Exams

In [47]:
X, y = get_exams_dataset()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) 

print("\nNaive Bayes Classifier:")
nbc_classifier = NBC_Categorical()
nbc_classifier.fit(X_train, y_train)
nbc_classifier.eval(X_test, y_test)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) 

print("\nID3:")
id3_tree = ID3()
id3_tree.fit(X_train, y_train)
id3_tree.eval(X_test, y_test)


Naive Bayes Classifier:
Accuracy: 0.2633333333333333

ID3:
Accuracy: 0.25


0.25

## E-commerce

In [25]:
X, y = get_ecommerce_dataset()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) 

print("\nNaive Bayes Classifier:")
nbc_classifier = NBC_Categorical()
nbc_classifier.fit(X_train, y_train)
nbc_classifier.eval(X_test, y_test)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1) 

print("\nID3:")
id3_tree = ID3()
id3_tree.fit(X_train, y_train)
id3_tree.eval(X_test, y_test)


Naive Bayes Classifier:
Accuracy: 0.6915151515151515

ID3:
Accuracy: 0.5757575757575758


0.5757575757575758

## Airline passenger satisfaction

In [29]:
(X_train, y_train), (X_test, y_test) = get_airline_dataset("train"), get_airline_dataset("test")

print("\nNaive Bayes Classifier:")
nbc_classifier = NBC_Categorical()
nbc_classifier.fit(X_train, y_train)
nbc_classifier.eval(X_test, y_test)

(X_train, y_train), (X_test, y_test) = get_airline_dataset("train"), get_airline_dataset("test")

print("\nID3:")
id3_tree = ID3()
id3_tree.fit(X_train, y_train)
id3_tree.eval(X_test, y_test)


Naive Bayes Classifier:


c:\Users\Kamil\Documents\sem8-mgr\UMA\projekt\uma-randomforest-with-nbc\experiments\..\models\NBC_Categorical.py:66: RuntimeWarning: invalid value encountered in double_scalars
  labels_probabilites[label] = probab/np.sum(list(probabilities.values()))


Accuracy: 0.8908992916538343

ID3:


KeyboardInterrupt: 